# "backtesting crypto with exit signal"
> "How to fetch and backtest crypto data using fastquant"

- toc: true
- branch: master
- badges: true
- comments: true
- author: Mikee Jazmines
- categories: [crypto, backtest]

<a href="https://colab.research.google.com/github/enzoampil/fastquant/blob/master/examples/2020-05-20-backtest_crypto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [45]:
# uncomment to install in colab
# !pip3 install fastquant --update
# or pip install git+https://www.github.com/enzoampil/fastquant.git@history

## fetch data from binance

### If a timestamp is given, it will return upto that timestamp

In [2]:
from fastquant import get_crypto_data

In [62]:
crypto = get_crypto_data("LUNA/USDT", 
                         "2022-01-23 00:00:00", 
                         "2022-02-23 00:00:00",
                         time_resolution='1h'
                        )

In [63]:
crypto.tail()

,open,high,low,close,volume
dt,,,,,
2022-02-22 20:00:00,53.84,54.34,53.38,53.70,197495.77
2022-02-22 21:00:00,53.69,53.95,53.03,53.04,135259.86
2022-02-22 22:00:00,53.04,53.51,52.60,53.25,125883.42
2022-02-22 23:00:00,53.25,54.95,53.16,54.74,258616.24
2022-02-23 00:00:00,54.74,55.95,54.54,55.41,401518.24


In [64]:
# Import modules
import backtrader as bt

# Import from package
from fastquant.strategies.base import BaseStrategy


class RSIStrategy(BaseStrategy):

    params = (("rsi_period", 14), ("rsi_upper", 70), ("rsi_lower", 30))

    def __init__(self):

        # Initialize global variables
        super().__init__()
        # Strategy level variables
        self.rsi_period = self.params.rsi_period
        self.rsi_upper = self.params.rsi_upper
        self.rsi_lower = self.params.rsi_lower

        if self.strategy_logging:
            print("===Strategy level arguments===")
            print("rsi_period :", self.rsi_period)
            print("rsi_upper :", self.rsi_upper)
            print("rsi_lower :", self.rsi_lower)
        self.rsi = bt.indicators.RelativeStrengthIndex(period=self.rsi_period, upperband=self.rsi_upper, lowerband=self.rsi_lower)

    def buy_signal(self):
        return self.rsi[0] < self.rsi_lower

    def sell_signal(self):
        return self.rsi[0] > self.rsi_upper

    def exit_long_signal(self):
        return self.rsi[0] >= 50


## run backtest using optimum values

In [65]:
from fastquant import backtest

In [66]:
import matplotlib as pl
pl.style.use("default")
pl.rcParams["figure.figsize"] = (9,5)

In [72]:
results, history = backtest(RSIStrategy, 
                               crypto, 
                            rsi_period = 20,
                            rsi_upper = 70,
                            rsi_lower = 30,
                               plot=False,
                               verbose=False,
                               return_history=True
                              )

In [74]:
indicators = history['indicators']

In [75]:
orders = history['orders']
orders

,strat_id,strat_name,dt,type,price,size,value,commission,pnl
0,0,rsi_period20_rsi_upper70_rsi_lower30,2022-01-28 12:00:00,buy,49.19,2030,99855.70,0.0,0.00
1,0,rsi_period20_rsi_upper70_rsi_lower30,2022-01-28 13:00:00,buy,48.39,2,96.78,0.0,0.00
2,0,rsi_period20_rsi_upper70_rsi_lower30,2022-01-29 08:00:00,sell,53.06,-2032,99952.48,0.0,7865.44
3,0,rsi_period20_rsi_upper70_rsi_lower30,2022-01-30 22:00:00,buy,46.04,2340,107733.60,0.0,0.00
4,0,rsi_period20_rsi_upper70_rsi_lower30,2022-01-31 01:00:00,buy,44.56,2,89.12,0.0,0.00
5,0,rsi_period20_rsi_upper70_rsi_lower30,2022-01-31 14:00:00,sell,47.47,-2342,107822.72,0.0,3352.02
6,0,rsi_period20_rsi_upper70_rsi_lower30,2022-02-11 01:00:00,buy,51.83,2143,111071.69,0.0,0.00
7,0,rsi_period20_rsi_upper70_rsi_lower30,2022-02-11 14:00:00,sell,54.57,-2143,111071.69,0.0,5871.82
8,0,rsi_period20_rsi_upper70_rsi_lower30,2022-02-11 23:00:00,buy,50.32,2324,116943.68,0.0,0.00
9,0,rsi_period20_rsi_upper70_rsi_lower30,2022-02-12 01:00:00,buy,49.89,2,99.78,0.0,0.00


The final value in `results` can be calculated from the `commission` and `pnl` (profit & loss) of all the closed (bought and sold) transactions in history:

In [76]:
r = results.squeeze()
r.final_value

130889.91000000003

In [77]:
r.init_cash + orders.pnl.sum() - orders.commission.sum()

130889.91000000002

In [80]:
profit = (r.final_value - 100000)/100000*100
print("Strategy")
print(f"Profit of asset: {profit}")

Strategy
Profit of asset: 30.889910000000032


In [81]:
print("Buy and hold")
print(f"Start price of asset: {crypto.iloc[0, 3]}")
print(f"End price of asset: {crypto.iloc[-1, 3]}")
print(f"Profit of asset: {(crypto.iloc[-1, 3] - crypto.iloc[0, 3])/crypto.iloc[0, 3]*100}")

Buy and hold
Start price of asset: 61.71
End price of asset: 55.41
Profit of asset: -10.209042294603798
